In [1]:
import pandas as pd

import csv
from time import sleep
from getpass import getpass

from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

In [2]:
driver = Chrome()

In [3]:
driver.get('https://www.twitter.com/login')

In [4]:
username = driver.find_element_by_xpath('//input[@name="session[username_or_email]"]')

In [5]:
username.send_keys('user')

In [ ]:
my_password = getpass()

In [7]:
password = driver.find_element_by_xpath('//input[@name="session[password]"]')

In [8]:
password.send_keys(my_password)

In [9]:
password.send_keys(Keys.RETURN)

In [12]:
search_input = driver.find_element_by_xpath('//input[@aria-label="Búsqueda"]')

In [13]:
#text
search_input.send_keys('#Bitcoin')

In [14]:
#enter
search_input.send_keys(Keys.RETURN)

In [15]:
driver.find_element_by_link_text('Más reciente').click()

In [19]:
cards = driver.find_elements_by_xpath('//div[@data-testid="tweet"]')

In [20]:
card = cards[0]

In [21]:
#username
card.find_element_by_xpath('./div[2]/div[1]//span').text

'Cryptoify'

In [22]:
#username
card.find_element_by_xpath('.//span').text

'Cryptoify'

In [23]:
#twitter_handle
card.find_element_by_xpath('.//span[contains(text(),"@")]').text

'@cryptoify'

In [24]:
#postdate
card.find_element_by_xpath('.//time').get_attribute('datetime')

'2021-05-26T18:18:10.000Z'

In [25]:
#content of tweet
comment = card.find_element_by_xpath('./div[2]/div[2]/div[1]').text
responding = card.find_element_by_xpath('./div[2]/div[2]/div[2]').text

In [26]:
comment + responding

'US Regulators Urgently Working on Joint #Cryptocurrency Regulation: Fed\'s Quarles Says It\'s \'High Priority\' – Regulation #Bitcoin NewsUS Regulators Urgently Working on Joint Cryptocurrency Regulation: Fed\'s Quarles Says It\'s \'High...\nU.S. government agencies are urgently working together on a joint regulatory framework for cryptocurrencies, Federal Reserve Vice Chair of Supervision Randal Quarles has confirmed. "We along with the...\nnews.bitcoin.com'

In [27]:
#reply count
card.find_element_by_xpath('.//div[@data-testid="reply"]').text

''

In [28]:
#retweet
card.find_element_by_xpath('.//div[@data-testid="retweet"]').text

''

In [29]:
#likes
card.find_element_by_xpath('.//div[@data-testid="like"]').text

''

In [30]:
def get_tweet_data(card):
    username = card.find_element_by_xpath('.//span').text
    handle = card.find_element_by_xpath('.//span[contains(text(),"@")]').text
    try:
        postdate = card.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    comment = card.find_element_by_xpath('./div[2]/div[2]/div[1]').text
    responding = card.find_element_by_xpath('./div[2]/div[2]/div[2]').text
    text = comment + responding
    
    reply_cnt = card.find_element_by_xpath('.//div[@data-testid="reply"]').text
    retweet_cnt = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    like_cnt = card.find_element_by_xpath('.//div[@data-testid="like"]').text
    
    tweet = (username, handle, postdate, text, reply_cnt, retweet_cnt, like_cnt)
    
    return tweet

In [31]:
get_tweet_data(card)

('Cryptoify',
 '@cryptoify',
 '2021-05-26T18:18:10.000Z',
 'US Regulators Urgently Working on Joint #Cryptocurrency Regulation: Fed\'s Quarles Says It\'s \'High Priority\' – Regulation #Bitcoin NewsUS Regulators Urgently Working on Joint Cryptocurrency Regulation: Fed\'s Quarles Says It\'s \'High...\nU.S. government agencies are urgently working together on a joint regulatory framework for cryptocurrencies, Federal Reserve Vice Chair of Supervision Randal Quarles has confirmed. "We along with the...\nnews.bitcoin.com',
 '',
 '',
 '')

In [32]:
tweet_data = []

for card in cards:
    data = get_tweet_data(card)
    if data:
        tweet_data.append(data)

In [33]:
tweet_data[0]

('Cryptoify',
 '@cryptoify',
 '2021-05-26T18:18:10.000Z',
 'US Regulators Urgently Working on Joint #Cryptocurrency Regulation: Fed\'s Quarles Says It\'s \'High Priority\' – Regulation #Bitcoin NewsUS Regulators Urgently Working on Joint Cryptocurrency Regulation: Fed\'s Quarles Says It\'s \'High...\nU.S. government agencies are urgently working together on a joint regulatory framework for cryptocurrencies, Federal Reserve Vice Chair of Supervision Randal Quarles has confirmed. "We along with the...\nnews.bitcoin.com',
 '',
 '',
 '')

In [35]:
data = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

while scrolling:
    page_cards = driver.find_elements_by_xpath('//div[@data-testid="tweet"]')
    for card in page_cards[-15:]:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)
            
    scroll_attempt = 0
    while True:
        # check scroll position
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(2)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1
            
            # end of scroll region
            if scroll_attempt >= 3:
                scrolling = False
                break
            else:
                sleep(2) # attempt another scroll
        else:
            last_position = curr_position
            break

# close the web driver
driver.close()

In [36]:
with open('scraped_tweets.csv', 'w', newline='', encoding='utf-8') as f:
    header = ['UserName', 'Handle', 'Timestamp', 'Text', 'Comments', 'Likes', 'Retweets']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)

In [37]:
df = pd.read_csv("scraped_tweets.csv")

In [39]:
df.shape

(52, 7)

In [41]:
df.head(10)

,UserName,Handle,Timestamp,Text,Comments,Likes,Retweets
0,Luke Green,@lukusdukus,2021-05-26T18:10:28.000Z,It's inevitable... #Bitcoin \n@AppleCitar Twee...,NaN,NaN,NaN
1,Dharmadip Rana,@DharmadipRana,2021-05-26T18:10:27.000Z,En respuesta a \n@BitcoinBro303PayPal to Let U...,NaN,NaN,NaN
2,(( ALBASHA )),@AL_CHART,2021-05-26T18:10:24.000Z,هل مهتم/ة بالعملات الرقميه ؟\n\n#بتكوين \n#Bit...,NaN,NaN,NaN
3,starz1111,@starz21580282,2021-05-26T18:10:24.000Z,Ignore the #FUD and keep on buying the #dip . ...,NaN,NaN,NaN
4,NaN,@bkatcap,2021-05-26T18:10:24.000Z,#bitcoin folks are the bestCitar Tweet\nnaiive...,NaN,NaN,NaN
5,Julien ROMAN,@JulienROMAN13,2021-05-26T18:10:22.000Z,Mega news: Apple recrute des experts en crypto...,4.0,10.0,51.0
6,Edd Gimenez,@eddgimenez,2021-05-26T18:10:19.000Z,#Bitcoin Is A Savings Account With An Average ...,NaN,NaN,NaN
7,News Sentiment Report,@NewsSentiment,2021-05-26T18:10:18.000Z,Analyzed 1277 articles. #Bitcoin #Covid_19News...,NaN,NaN,NaN
8,Trepy ₿,@Trepyoficial,2021-05-26T18:10:03.000Z,Instagram acaba de activar la función para oc...,NaN,NaN,NaN
9,aWebAnalysis | Crypto,@aWebAnalysis,2021-05-26T18:10:03.000Z,"Bitcoin BTC Current Price:\n$38,920.17\n1 Hour...",NaN,NaN,NaN
